# Preprocess MIMIC-III data for long-term heart failure prediction
General workflow:
1. get the features we want
2. process categorical features
3. process continuous features with lower recording frequencing
4. process continuous features with higher recording frequencing
5. combine it all.

### General set up and filtering

#### Load files and set up

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mrmr import mrmr_classif
from sklearn.impute import KNNImputer
import os

# column names
ID = 'ID'
TIME = 't'
VAR = 'variable_name'
VAL = 'variable_value'
HF_LABEL = 'HF_LABEL'

# Set the default plot size
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_style("whitegrid")

In [ ]:
# load data
df = pd.read_pickle('data/input_data.p')
labels = pd.read_csv('data/HF_240.0h.csv')
df = pd.merge(df, labels[[ID, HF_LABEL]], on=ID) # map the patient heart failure label to the data

# get names of features from dictionary tables
d_items = pd.read_csv('data/D_ITEMS.csv')
d_lab = pd.read_csv('data/D_LABITEMS.csv')

# load in additional data
icu_stays = pd.read_csv('data/ICUSTAYS.csv')
dx = pd.read_csv('data/DIAGNOSES_ICD.csv')
rx = pd.read_csv('data/PRESCRIPTIONS.csv')

# map the ICUSTAY_ID to diagnoses
id_map = icu_stays[icu_stays['ICUSTAY_ID'].isin(df[ID].unique())][['HADM_ID','ICUSTAY_ID']]
dx = pd.merge(dx, id_map, on='HADM_ID')

# format columns
dx = dx.rename(columns={
    'ICUSTAY_ID': ID,
    'ICD9_CODE' : VAR
})

dx[VAL] = 1
dx[TIME] = np.nan

dx = dx[[ID, TIME, VAR, VAL]]
dx = pd.merge(dx, labels[[ID, HF_LABEL]], on=ID)
dx[VAR] = dx[VAR].apply(lambda x: 'ICD9: ' + x)


rx = rx.rename(columns={
    'ICUSTAY_ID': ID,
    'DRUG' : VAR
})

rx[VAL] = 1
rx[TIME] = np.nan

rx = rx[[ID, TIME, VAR, VAL]]
rx = pd.merge(rx, labels[[ID, HF_LABEL]], on=ID)
rx[VAR] = rx[VAR].apply(lambda x: 'RX: ' + x)

# combine data
data = pd.concat([df, dx, rx])

#### Get features from literature

In [ ]:
'''
Prediction model of in-hospital mortality in intensive care unit patients with heart failure: 
machine learning-based, retrospective analysis of the MIMIC-III database
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8311359/
'''

demographics1 = ['AGE', 'GENDER', 'ETHNICITY', 'Weight', 'Height']
vital_signs = ['HR', 'SysBP', 'DiaBP', 220181, 'RR', 'Temperature', 'SpO2']
labs = [51221, 51493, 51222, 51250, 51277, 51265, 51274, 51237, 51006, 50931, 50971, 50983,
        50893, 50902, 50960, 50868, 50882]

'''
Guo et al. An evaluation of time series summary statistics as features for clinical prediction tasks
'''
item_ids = [220739, 223901, 223900, 223835]
lab_ids = [51301, 50971, 50822, 50821, 50882, 50983, 51006, 50885]
demographic2 = ['AGE', 'ADMISSION_TYPE']

'''
Define our features types.
These I assessed by hand.
'''
categorical = ['AGE', 'GENDER', 'ADMISSION_TYPE', 'ETHNICITY', 220739, 223900, 223901]
continuous = ['HR', 'SysBP', 'DiaBP', 220181, 'RR', 'SpO2', 'Weight', 'Height', 
              51221, 51222, 51250, 51277, 51265, 51274, 51237, 51006, 50931, 50971, 50983, 51493,
              50893, 50902, 50960, 50868, 50882, 223835, 51301, 50971, 50822, 50821, 50882,
              50983, 51006, 50885]

# make them strings (because the variable_name column in df is)
categorical = [str(x) for x in categorical]
continuous = [str(x) for x in continuous]

# combine
all_features = categorical + continuous

#### Identify train and test sets

In [ ]:
# split data into training and test sets by patient ID
patient_ids = data[ID].unique()
train_ids, test_ids = train_test_split(patient_ids, test_size=0.3, random_state=0)

In [ ]:
# filter out rare features in training patients
data_train = data[data[ID].isin(train_ids)]
n_patients = data_train[ID].nunique()
a = pd.DataFrame(data_train[[ID,VAR]].drop_duplicates().groupby(VAR)[ID].nunique()).reset_index()
a.columns = [VAR, 'n_patients']
a['n_patients'] = a['n_patients'] / n_patients
a_rx_dx = a[a[VAR].str.contains('RX:|ICD9:')]
a = a[~a[VAR].str.contains('RX:|ICD9:')]

rare_features = a[a['n_patients'] <= 0.7][VAR].values.tolist()
rare_dx_rx = a_rx_dx[a_rx_dx['n_patients'] <= 0.01][VAR].values.tolist()

data_filtered = data[data[VAR].isin(rare_features + rare_dx_rx) == False]

#### Separate continuous and categorical features

In [ ]:
# separate categorical features
categorical.extend(data_filtered[data_filtered[VAR].str.contains('RX:|ICD9:')][VAR].unique().tolist())
df_cat = data_filtered[data_filtered['variable_name'].isin(categorical)].reset_index(drop=True)
df_cat = df_cat.drop_duplicates()

# separate continuous features
df_cont = data_filtered[data_filtered['variable_name'].isin(continuous)].reset_index(drop=True)

### Process Categorical Features

In [ ]:
# see how many times each categorical feature is recorded for each patient
cat_t_count = pd.DataFrame(df_cat.groupby(['ID','variable_name'])['t'].nunique()).reset_index()

#### Encode demographics

In [ ]:
# get and format demographics
df_cat_demo = df_cat[df_cat['t'].isna()]

# get ICD supergroups
df_dx = df_cat_demo[df_cat_demo['variable_name'].str.contains('ICD9:')]
df_dx['variable_name'] = df_dx['variable_name'].apply(lambda x: x.split('ICD9: ')[1])
df_dx['variable_name'] = 'ICD9: ' + df_dx['variable_name'].str.slice(0,3)
df_dx = df_dx.drop_duplicates()
df_cat_demo = pd.concat([df_cat_demo, df_dx])
df_cat_demo = df_cat_demo.drop_duplicates()

df_cat_demo = df_cat_demo.pivot(index='ID', columns='variable_name', values='variable_value')

# encode admission types on a scale
admission_type_scale = {'ELECTIVE' : 1, 'URGENT' : 2, 'EMERGENCY' : 3}
df_cat_demo['ADMISSION_TYPE'] = df_cat_demo['ADMISSION_TYPE'].map(admission_type_scale)

# encode gender
gender_scale = {'M' : 1, 'F' : 0}
df_cat_demo['GENDER'] = df_cat_demo['GENDER'].map(gender_scale)
df_cat_demo.rename(columns={'GENDER' : 'Female'}, inplace=True)

# one hot encode ethnicity
df_cat_demo = pd.get_dummies(df_cat_demo, columns=['ETHNICITY'])

#### Encode categorical features according to Glascow Coma Scale
https://www.cdc.gov/masstrauma/resources/gcs.pdf

In [ ]:
eye_opening_220739_scale = {'None' : 1, 'To Pain' : 2, 'To Speech' : 3, 'Spontaneously' : 4}
verbal_223900_scale = {'No Response' : 1, 'No Response-ETT' : 1, 'Incomprehensible sounds' : 2, 'Inappropriate Words' : 3, 'Confused' : 4, 'Oriented' : 5}
motor_223901_scale = {'No response' : 1, 'Abnormal extension' : 2, 'Abnormal Flexion' : 3, 'Flex-withdraws' : 4, 'Localizes Pain' : 5, 'Obeys Commands' : 6}

In [ ]:
idx = df_cat[df_cat['variable_name'] == '220739'].index
df_cat.loc[idx, 'variable_value'] = df_cat.loc[idx, 'variable_value'].map(eye_opening_220739_scale).values

idx = df_cat[df_cat['variable_name'] == '223900'].index
df_cat.loc[idx, 'variable_value'] = df_cat.loc[idx, 'variable_value'].map(verbal_223900_scale).values

idx = df_cat[df_cat['variable_name'] == '223901'].index
df_cat.loc[idx, 'variable_value'] = df_cat.loc[idx, 'variable_value'].map(motor_223901_scale).values

In [ ]:
# keep mean GCS score
df_cat_gcs = df_cat[df_cat['variable_name'].isin(['220739', '223900', '223901'])]
df_cat_gcs = pd.DataFrame(df_cat_gcs.groupby(['ID','variable_name'])['variable_value'].mean()).reset_index()
df_cat_gcs = df_cat_gcs.pivot(index='ID', columns='variable_name', values='variable_value') # pivot

#### Combine

In [ ]:
df_cat_demo = df_cat_demo.fillna(0)

df_cat_processed = pd.concat([df_cat_demo, df_cat_gcs], axis=1)

### Process Continuous Features

In [ ]:
x = df_cont[VAL].str.contains('[a-zA-Z]', regex=True).fillna(False)
df_cont = df_cont[~x]
df_cont[VAL] = df_cont[VAL].str.strip()

# Separate continuous features that are frequently sampled

# see how many times each continuous feature is recorded for each patient
cont_t_count = pd.DataFrame(df_cont.groupby(['ID','variable_name'])['t'].nunique()).reset_index()


# based on this plot, I am only going to calculate all the summary statistics from features with more than 20 time points on average per patient
# for all those with <20 times points per patient on average, I will just calculate a mean

freq_cutoff = 20
cont_mean_t_count = pd.DataFrame(df_cont.groupby(['ID','variable_name'])['t'].nunique().groupby('variable_name').mean()).reset_index().sort_values('t', ascending=False)
high_freq_features = cont_mean_t_count[cont_mean_t_count['t'] >= freq_cutoff]['variable_name'].tolist()
low_freq_features = cont_mean_t_count[cont_mean_t_count['t'] < freq_cutoff]['variable_name'].tolist()
all_cont_features = high_freq_features + low_freq_features

In [ ]:
# reformat values as floats
df_cont_low_freq = df_cont[df_cont['variable_name'].isin(low_freq_features)]

# convert values to floats (rbc = 51493)
def to_float(x):
    if isinstance(x, float) or isinstance(x, int):
        return x
    elif '-' in x:
        return np.mean([float(y) for y in x.split('-')])
    elif '>' in x or '<' in x:
        return float(x[1:])
    elif x == ' ' or 'ERROR' in x or 'UNABLE' in x or x.count('.') > 1 or x == '':
        return np.nan
    elif 'GREATER' in x:
        return float(x.split(' ')[-1])
    elif ': ' in x:
        return float(x.split(': ')[-1])
    else:
        return float(x)

df_cont_low_freq['variable_value'] = df_cont_low_freq['variable_value'].apply(lambda x: to_float(x))

# get mean value for each patient and pivot
df_cont_low_freq = pd.DataFrame(df_cont_low_freq.groupby(['ID','variable_name'])['variable_value'].mean()).reset_index()
df_cont_low_freq = df_cont_low_freq.pivot(index='ID', columns='variable_name', values='variable_value')

#### Compute summary statistics for continuous features with high sampling frequency

In [ ]:
# reformat values as floats
df_cont_high_freq = df_cont[df_cont['variable_name'].isin(all_cont_features)]
df_cont_high_freq['variable_value'] = df_cont_high_freq['variable_value'].apply(lambda x: to_float(x))

# compute mean and stdv
df_cont_high_freq_mean = pd.DataFrame(df_cont_high_freq.groupby(['ID','variable_name'])['variable_value'].mean()).reset_index()
df_cont_high_freq_std = pd.DataFrame(df_cont_high_freq.groupby(['ID','variable_name'])['variable_value'].std()).reset_index()
df_cont_high_freq_mean['variable_name'] = df_cont_high_freq_mean['variable_name'] + '_mean'
df_cont_high_freq_std['variable_name'] = df_cont_high_freq_std['variable_name'] + '_std'
df_cont_high_freq_mean = df_cont_high_freq_mean.pivot(index='ID', columns='variable_name', values='variable_value')
df_cont_high_freq_std = df_cont_high_freq_std.pivot(index='ID', columns='variable_name', values='variable_value')


df_cont_high_freq = pd.concat([df_cont_high_freq_mean, df_cont_high_freq_std], axis=1) # df_cont_high_freq_variation, df_cont_high_freq_iqr

### Combine all data

In [ ]:
# combine all data
df_processed = pd.concat([df_cont_high_freq, df_cat_processed], axis=1) #df_cont_low_freq

In [ ]:
import icd9cms.icd9 as icd9

In [ ]:
# get feature names
cols = df_processed.columns
d_icd = pd.read_csv('data/D_ICD_DIAGNOSES.csv')

for col in cols:

    # init vars
    id = None
    label = None
    has_stat_suffix = False
    icd = False

    # check if feature is ID we can map to feature name
    if col.isdigit():
        id = int(col)
    elif '_' in col:
        feature = col.split('_')[0]
        if feature.isdigit():
            id = int(feature)
            has_stat_suffix = True
    elif 'ICD9:' in col:
        id = col.split('ICD9: ')[-1]
        icd = True

    # find feature label in tables and update
    if id is not None:
        if icd:
            if id in d_icd['ICD9_CODE'].values:
                label = d_icd[d_icd['ICD9_CODE'] == id]['LONG_TITLE'].values[0]
                label = 'DX: ' + label + f' ({id})'
            elif icd9.search(id):
                label = 'DX: ' + icd9.search(id).short_desc + f' ({id})'
            icd = False
        if id in d_items['ITEMID'].values:
            label = d_items[d_items['ITEMID'] == id]['LABEL'].values[0]
        elif id in d_lab['ITEMID'].values:
            label = d_lab[d_lab['ITEMID'] == id]['LABEL'].values[0]
        
        if label is not None:
            if has_stat_suffix:
                df_processed.rename(columns={col: label + '_' + col.split('_')[1]}, inplace=True)
            else:
                df_processed.rename(columns={col: label}, inplace=True)

In [ ]:
# move and format ID column
df_processed = df_processed.reset_index()
df_processed['ID'] = df_processed['ID'].astype(int)

# get target variable
labels['ID'] = labels['ID'].astype(int)
df_processed = pd.merge(df_processed, labels[['ID','HF_LABEL']], on='ID')

In [ ]:
# remove diagnosis and medication features
leaky_cols = ['Do not resuscitate status (V4986)','Encounter for palliative care (V667)', 'Convalescence and palliative care (V66)']
df_processed = df_processed[df_processed.columns[~df_processed.columns.isin(leaky_cols)]]
# df_processed = df_processed[df_processed.columns[~df_processed.columns.str.contains('RX:|DX:')]]


#### Split into train and test set

In [ ]:
# split data into training and test sets by patient ID
x_train = df_processed[df_processed[ID].isin(train_ids)].drop(columns=[ID,HF_LABEL])
x_test = df_processed[df_processed[ID].isin(test_ids)].drop(columns=[ID,HF_LABEL])
y_train = df_processed[HF_LABEL][df_processed[ID].isin(train_ids)]
y_test = df_processed[HF_LABEL][df_processed[ID].isin(test_ids)]

# get complete processed dataset with IDs, HF labels, and train/test labels
df_processed['TRAIN'] = df_processed[ID].isin(train_ids) * 1

x_train = x_train.astype(float)
x_test = x_test.astype(float)

#### Feature selection and imputation

In [ ]:
# feature selection
selected_features, relvence_scores, redundance_scores = mrmr_classif(x_train.astype(float), y_train, K=30, return_scores=True, show_progress=True)
x_train = x_train[selected_features]
x_test = x_test[selected_features]

# impute missing values with knn
imputer = KNNImputer(n_neighbors=9, weights='distance')
x_train = pd.DataFrame(imputer.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(imputer.transform(x_test), columns=x_test.columns)

#### Write to file

In [ ]:
out_dir = 'data_nolowfreq_statsmeanstd_mrmr30_knn9distweight_noleakage'
os.makedirs(out_dir, exist_ok=True)
x_train.to_csv(f'{out_dir}/x_train.csv', index=False)
x_test.to_csv(f'{out_dir}/x_test.csv', index=False)
y_train.to_csv(f'{out_dir}/y_train.csv', index=False)
y_test.to_csv(f'{out_dir}/y_test.csv', index=False)
df_processed.to_csv(f'{out_dir}/df_processed.csv', index=False)
data_filtered.to_csv(f'{out_dir}/data_filtered.csv', index=False)